
# LNG Optimisation — Exploration Notebook (Clean & Runnable)

This notebook gives you a **single, end‑to‑end workflow**:
1. Load **real market data** (robust Excel parsing, JKM-friendly).
2. Build a **unit-economics & risk‑aware profit table** per destination.
3. Solve an **allocation LP** with PuLP to maximise (risk‑adjusted) profit under capacity & supply constraints.
4. Run **scenarios** and visualise allocations and P&L.

> **Note:** Charts are rendered with matplotlib only (no seaborn), and each chart is on its own figure as requested.


## 0) Setup & Installs

In [ ]:

# If PuLP is not installed, uncomment:
# %pip install pulp openpyxl --quiet

import warnings
warnings.filterwarnings("ignore")

from dataclasses import dataclass, field
from pathlib import Path
from typing import Dict, Iterable, List, Optional, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pulp


## 1) Configuration

In [ ]:

@dataclass
class LoaderConfig:
    data_folder: Path = Path("data")
    # Explicitly use the first sheet by default to avoid dict-of-DFs from pandas:
    sheet: Optional[str | int] = 0
    excel_engine: Optional[str] = "openpyxl"

    historical_files: Dict[str, str] = field(default_factory=lambda: {
        "jkm_historical": "JKM Spot LNG Historical (Extracted 23Sep25).xlsx",
        "henry_hub_historical": "Henry Hub Historical (Extracted 23Sep25).xlsx",
        "ttf_historical": "TTF Historical (Extracted 23Sep25).xlsx",
    })

    forward_files: Dict[str, str] = field(default_factory=lambda: {
        "jkm_forward": "JKM Spot LNG Forward (Extracted 23Sep25).xlsx",
        "henry_hub_forward": "Henry Hub Forward (Extracted 23Sep25).xlsx",
        "ttf_forward": "TTF Forward (Extracted 23Sep25).xlsx",
    })

cfg = LoaderConfig()
print('Looking for data in:', cfg.data_folder.resolve())


## 2) Robust Excel Loader (JKM-friendly)

In [ ]:

import re

class LNGLoader:
    def __init__(self, cfg: LoaderConfig):
        self.cfg = cfg
        self.historical_data: Dict[str, pd.DataFrame] = {}
        self.forward_data: Dict[str, pd.DataFrame] = {}
        self.metrics: Dict[str, Dict] = {}

    # ---- IO helpers ----
    def _read_sheet(self, path: Path, header=None) -> pd.DataFrame:
        df = pd.read_excel(path, sheet_name=self.cfg.sheet, engine=self.cfg.excel_engine, header=header)
        if isinstance(df, dict):  # safety
            key = list(df.keys())[0]
            df = df[key]
        return df

    @staticmethod
    def _clean_str(x) -> str:
        if x is None or (isinstance(x, float) and np.isnan(x)): return ""
        return str(x).strip().replace("\xa0"," ").lower()

    def _find_row_containing(self, df: pd.DataFrame, patterns: Iterable[str], anywhere: bool = False) -> Optional[int]:
        pats = [re.compile(p, flags=re.I) for p in patterns]
        for i in range(len(df)):
            row = df.iloc[i]
            cells = row.tolist() if anywhere else [row.iloc[0]]
            text = " ".join(self._clean_str(c) for c in cells)
            if any(p.search(text) for p in pats):
                return i
        return None

    def _find_header_row(self, df: pd.DataFrame, start: int, window: int, contains: List[str]) -> Optional[int]:
        wants = [self._clean_str(x) for x in contains]
        for i in range(start, min(start+window, len(df))):
            row_text = " ".join(self._clean_str(x) for x in df.iloc[i].tolist())
            if all(tok in row_text for tok in wants):
                return i
        return None

    def _find_table_end(self, df: pd.DataFrame, start: int, stop_keywords: List[str]) -> int:
        stops = [self._clean_str(k) for k in stop_keywords]
        for i in range(start, len(df)):
            first = self._clean_str(df.iloc[i].iloc[0])
            if first == "" or any(k in first for k in stops):
                return i
        return len(df)

    # ---- Cleaners ----
    def _clean_generic_price(self, df: pd.DataFrame, source_name: str) -> pd.DataFrame:
        df2 = df.copy()
        df2.replace(r"^\s*$", np.nan, regex=True, inplace=True)
        df2.dropna(how="all", inplace=True)
        df2.columns = [f"col{j}" for j in range(df2.shape[1])]

        # Attempt to find date & price columns by name; fallback to first two
        dcol = next((c for c in df2.columns if re.search(r"date|exchange", str(c), re.I)), df2.columns[0])
        pcol = next((c for c in df2.columns if re.search(r"close|price|settle|usd", str(c), re.I)), df2.columns[1])

        out = df2[[dcol, pcol]].rename(columns={dcol:"date", pcol:"price"})
        out["date"] = pd.to_datetime(out["date"], errors="coerce")
        out["price"] = pd.to_numeric(out["price"], errors="coerce")
        out.dropna(subset=["date","price"], inplace=True)
        out.sort_values("date", inplace=True)
        out.set_index("date", inplace=True)
        out["daily_return"] = out["price"].pct_change()
        out["volatility_30d"] = out["daily_return"].rolling(30).std() * np.sqrt(252)
        out["source"] = source_name
        return out

    def _clean_jkm(self, df: pd.DataFrame) -> pd.DataFrame:
        # 1) find "JKMc1 History" anchor anywhere in the row
        anchor = self._find_row_containing(df, [r"jkmc?1\s+history"], anywhere=True)
        if anchor is None:
            return self._clean_generic_price(df, "jkm_historical")

        # 2) find header within next rows (must contain both terms)
        header = self._find_header_row(df, start=anchor, window=25, contains=["exchange date", "close"])
        if header is None:
            return self._clean_generic_price(df, "jkm_historical")

        # 3) slice until blank/new section
        end = self._find_table_end(df, start=header+1, stop_keywords=["statistics","vap","summary"])
        body = df.iloc[header+1:end].copy()
        cols = [self._clean_str(x) or f"col{j}" for j,x in enumerate(df.iloc[header].tolist())]
        body.columns = cols

        # map columns
        dcol = next((c for c in body.columns if "exchange date" in c or "date" in c), None)
        pcol = next((c for c in body.columns if "close" in c or "price" in c or "settle" in c), None)
        if dcol is None or pcol is None:
            return self._clean_generic_price(df, "jkm_historical")

        out = body[[dcol,pcol]].rename(columns={dcol:"date", pcol:"price"})
        out["date"] = pd.to_datetime(out["date"], errors="coerce")
        out["price"] = pd.to_numeric(out["price"], errors="coerce")
        out.dropna(subset=["date","price"], inplace=True)
        out.sort_values("date", inplace=True)
        out.set_index("date", inplace=True)
        out["daily_return"] = out["price"].pct_change()
        out["volatility_30d"] = out["daily_return"].rolling(30).std() * np.sqrt(252)
        out["source"] = "jkm_historical"
        return out

    # ---- Public load ----
    def load_all(self) -> None:
        print("📈 Loading historical price data...")
        for key, fname in cfg.historical_files.items():
            fpath = cfg.data_folder / fname
            print(f"  • {key}: {fpath.name}")
            if not fpath.exists():
                print("    ↳ ⚠️ Not found; skipping.")
                continue
            try:
                df_raw = self._read_sheet(fpath, header=None)
                df_clean = self._clean_jkm(df_raw) if "jkm" in key.lower() else self._clean_generic_price(df_raw, key)
                if df_clean.empty:
                    print("    ↳ ❌ No rows after cleaning.")
                else:
                    self.historical_data[key] = df_clean
                    print(f"    ↳ ✅ {len(df_clean)} rows.")
            except Exception as e:
                print(f"    ↳ ❌ Error: {e}")

        print("📈 Loading forward curve data...")
        for key, fname in cfg.forward_files.items():
            fpath = cfg.data_folder / fname
            print(f"  • {key}: {fpath.name}")
            if not fpath.exists():
                print("    ↳ ⚠️ Not found; skipping.")
                continue
            try:
                df_raw = self._read_sheet(fpath, header=0)
                df_raw.replace(r"^\s*$", np.nan, regex=True, inplace=True)
                df_raw.dropna(how="all", inplace=True)
                self.forward_data[key] = df_raw
                print(f"    ↳ ✅ {len(df_raw)} rows.")
            except Exception as e:
                print(f"    ↳ ❌ Error: {e}")

    def latest_prices(self) -> Dict[str, float]:
        out = {}
        for name, df in self.historical_data.items():
            if df.empty: 
                continue
            latest = float(df["price"].iloc[-1])
            n = name.lower()
            if "jkm" in n: out["JP"] = latest
            elif "ttf" in n: out["SLNG"] = latest * 1.1
            elif "henry_hub" in n: out["CN"] = latest * 1.5
        return out


## 3) Sample Ports & Assumptions (fallback if no files found)

In [ ]:

# If you already have the CSV/JSON from the Streamlit scaffold, you can reuse those.
# Otherwise, we'll define small inline defaults so the notebook runs anywhere.

ports_df = pd.DataFrame([
    {"name":"Singapore","code":"SLNG","distance_nm":600,"monthly_capacity_cargo":10,"handling_fee_per_unit":0.25},
    {"name":"Japan (JKM)","code":"JP","distance_nm":1800,"monthly_capacity_cargo":12,"handling_fee_per_unit":0.30},
    {"name":"China","code":"CN","distance_nm":1500,"monthly_capacity_cargo":9,"handling_fee_per_unit":0.28},
])

assumptions = {
    "supply_cargo_units": 20,
    "boiloff_rate_per_1000nm": 0.0015,
    "freight_cost_per_nm_per_unit": 0.02,
    "variable_cost_per_unit": 5.0,
    "carbon_cost_per_unit": 0.0,
}

# If loader pulls real prices, those will override this map.
fallback_prices = {"SLNG": 12.0, "JP": 14.0, "CN": 11.0}

ports_df


## 4) Unit Economics + LP Optimiser

In [ ]:

def delivered_fraction(distance_nm: float, boiloff_rate_per_1000nm: float) -> float:
    loss = (distance_nm/1000.0) * boiloff_rate_per_1000nm
    return max(0.0, 1.0 - loss)

def unit_profit(price_per_unit, distance_nm, handling_fee, a):
    delivered = delivered_fraction(distance_nm, a["boiloff_rate_per_1000nm"])
    revenue = price_per_unit * delivered
    freight = a["freight_cost_per_nm_per_unit"] * distance_nm
    costs = a["variable_cost_per_unit"] + handling_fee + freight + a.get("carbon_cost_per_unit",0.0)
    return revenue - costs, delivered

def build_profit_table(ports_df: pd.DataFrame, price_map: Dict[str, float], a: dict) -> pd.DataFrame:
    rows = []
    for _, r in ports_df.iterrows():
        code = r["code"]
        price = price_map.get(code, np.nan)
        up, delivered = unit_profit(price, r["distance_nm"], r["handling_fee_per_unit"], a)
        rows.append({
            "code": code,
            "name": r["name"],
            "distance_nm": r["distance_nm"],
            "monthly_capacity_cargo": r["monthly_capacity_cargo"],
            "price_per_unit": price,
            "unit_profit": up,
            "delivered_fraction": delivered
        })
    return pd.DataFrame(rows)

def optimise_allocation(unit_tbl: pd.DataFrame, total_supply_units: float) -> Dict:
    codes = unit_tbl["code"].tolist()
    profit = {row["code"]: float(row["unit_profit"]) for _, row in unit_tbl.iterrows()}
    capacity = {row["code"]: float(row["monthly_capacity_cargo"]) for _, row in unit_tbl.iterrows()}
    m = pulp.LpProblem("lng_allocation", pulp.LpMaximize)
    x = pulp.LpVariable.dicts("alloc", codes, lowBound=0)
    # objective
    m += pulp.lpSum([profit[c]*x[c] for c in codes])
    # constraints
    m += pulp.lpSum([x[c] for c in codes]) <= float(total_supply_units), "Supply"
    for c in codes:
        m += x[c] <= capacity[c], f"Cap_{c}"
    m.solve(pulp.PULP_CBC_CMD(msg=False))
    alloc = {c: float(x[c].value() or 0.0) for c in codes}
    return {"allocation": alloc, "objective": float(pulp.value(m.objective))}


## 5) Load Market Files (if present) and Build Table

In [ ]:

loader = LNGLoader(cfg)
loader.load_all()

price_map = fallback_prices.copy()
price_map.update(loader.latest_prices())  # real prices override

unit_tbl = build_profit_table(ports_df, price_map, assumptions)
unit_tbl


## 6) Solve and Visualise

In [ ]:

res = optimise_allocation(unit_tbl, assumptions["supply_cargo_units"])
alloc = pd.DataFrame([{"code":k, "allocation":v} for k,v in res["allocation"].items()]).merge(
    unit_tbl[["code","name","unit_profit","delivered_fraction"]], on="code"
).sort_values("allocation", ascending=False)

print("Expected Profit (objective): ${:,.0f}".format(res["objective"]))

# Plot: Allocation
plt.figure(figsize=(7,4))
plt.bar(alloc["name"], alloc["allocation"])
plt.title("Optimal Allocation (cargo units)")
plt.xlabel("Destination")
plt.ylabel("Units")
plt.tight_layout()
plt.show()

# Plot: Unit profit
plt.figure(figsize=(7,4))
plt.bar(unit_tbl["name"], unit_tbl["unit_profit"])
plt.title("Unit Profit by Destination (USD / loaded unit)")
plt.xlabel("Destination")
plt.ylabel("Unit Profit")
plt.tight_layout()
plt.show()

alloc


## 7) Scenario Analysis (Cold Snap, SLNG Outage)

In [ ]:

def run_scenario(name: str, price_shocks=None, capacity_mult=None):
    price_shocks = price_shocks or {}
    capacity_mult = capacity_mult or {}

    pm = price_map.copy()
    for k,v in price_shocks.items():
        if k in pm:
            pm[k] += v

    pf = ports_df.copy()
    pf["monthly_capacity_cargo"] = pf.apply(lambda r: r["monthly_capacity_cargo"] * capacity_mult.get(r["code"],1.0), axis=1)

    tbl = build_profit_table(pf, pm, assumptions)
    res = optimise_allocation(tbl, assumptions["supply_cargo_units"])

    delivered = sum(tbl.set_index("code").loc[c, "delivered_fraction"] * x for c,x in res["allocation"].items())
    return name, res["objective"], res["allocation"], tbl, delivered

scenarios = [
    ("Base", {}, {}),
    ("Cold snap NE Asia", {"JP": 3.0, "CN": 1.5}, {}),
    ("SLNG outage", {}, {"SLNG": 0.5}),
]

rows = []
scenario_tables = {}
for name, shocks, caps in scenarios:
    nm, obj, alloc_map, tbl, delivered = run_scenario(name, shocks, caps)
    scenario_tables[name] = (alloc_map, tbl)
    rows.append({"scenario": nm, "profit": obj, "delivered_units": delivered})

sc_df = pd.DataFrame(rows)
sc_df


### Scenario Profit Comparison

In [ ]:

plt.figure(figsize=(7,4))
plt.bar(sc_df["scenario"], sc_df["profit"])
plt.title("Scenario Comparison — Profit")
plt.xlabel("Scenario")
plt.ylabel("Profit (USD)")
plt.tight_layout()
plt.show()


## 8) Summary

In [ ]:

print("Price map used:", price_map)
print("\nTop-line:")
for _, r in sc_df.iterrows():
    print(f"  {r['scenario']}: profit=${r['profit']:,.0f}, delivered≈{r['delivered_units']:.2f} units")
